In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm

In [7]:
# --- Configuration ---
base_folder = os.path.join("/media", "ssudhakar", "DATA_10TB", "data_solver", "newrunGT")
if not os.path.exists(base_folder):
    raise AssertionError("No such folder: base_folder")

base_path = Path(base_folder)
scheme_map = {'1': 'RK1', '2': 'RK2', '3': 'RK3', '4': 'RK4', '5': 'AB2', '6': 'AB2RK2'}
grid_map = {'0': 128, '1': 256, '2': 512}
precision_map = {'2': '64bit', '3': '128bit'}

In [8]:
mkey = {}

for s in ['1', '2', '3', '4', '5', '6']:  # schemes
    scheme = scheme_map[s]
    for g in ['0', '1', '2']:  # grid sizes
        grid_size = grid_map[g]
        for p in ['2', '3']:  # precisions
            precision = precision_map[p].replace('-', '')  
            xyz = f"{s}{g}{p}"
            full_key = f"{scheme}_G{grid_size}_P{precision}"
            mkey[xyz] = full_key

svar1 = [ "dens", "pres", "temp", "u",  # General physical variables
          "h2", "h", "o", "o2", "oh", "h2o", "ho2", "h2o2", "ar", "n2"]  # Chemical species


svar2 = [ "Density ($kg/m^3$)", "Pressure ($N/m^2$)", "Temperature ($K$)", "Velocity - u - ($m/s$)",  # General physical variables
          "Hydrogen ($H_2$)", "Nascent Hydrogen ($H$)", "Nascent Oxygen ($O$)", "Oxygen ($O_2$)", "Hydroxide ($OH$)", 
          "Water ($H_2O$)", "Hydroperoxyl radical ($HO_2$)", "Hydrogen Peroxide ($H_2O_2$)", "Argon ($Ar$)", "Nitrogen ($N_2$)"]  # Chemical species

grid_128_GT = np.linspace(0, 5, 128, endpoint=False)
grid_256_GT = np.linspace(0, 5, 256, endpoint=False)
grid_512_GT = np.linspace(0, 5, 512, endpoint=False)
grids = [grid_128_GT, grid_256_GT, grid_512_GT]
gridstrs = ['$Nx_{128}$', '$Nx_{256}$', '$Nx_{512}$']

stride256 = 2
stride512 = 4

In [9]:
folder_name_ref = os.path.join("/media", "ssudhakar", "DATA_10TB", "data_solver", "newrunGT", "423GT")
xyz_ref = '423'
# print(xyz_ref)

data_ref = {}

scheme = scheme_map[xyz_ref[0]]
grid_size = grid_map[xyz_ref[1]]
precision = precision_map.get(xyz_ref[2], None)

key = f"{scheme}_G{grid_size}_P{precision}"
colnames = ["time"] + [f"col{col_num:04d}" for col_num in range(1, grid_size + 2)]


var_dict = {}
for var in tqdm([ "dens", "pres", "temp", "u",  # General physical variables
                "h2", "h", "o", "o2", "oh", "h2o", "ho2", "h2o2", "ar", "n2"]):
    try:  # Chemical species
        fname = os.path.join(folder_name_ref, f"output_sol_{var}_{xyz_ref}GT.csv")
        df = pd.read_csv(fname, header=None, names=colnames, dtype=str, nrows=8000).astype(np.longdouble)
        var_dict[var] = df
    except: 
        print(var)
        raise AssertionError
data_ref[key] = var_dict

100%|██████████| 14/14 [00:13<00:00,  1.02it/s]


In [11]:
available_cases = []
folders = sorted(base_path.glob('*GT'))
for folder in folders:
    available_cases.append(folder.name.split('G')[0])

print(available_cases)
print(f"The number of cases are: {len(available_cases)}")

['102', '103', '112', '113', '122', '123', '202', '203', '212', '213', '222', '223', '302', '303', '312', '313', '322', '323', '402', '403', '412', '413', '422', '423', '502', '503', '512', '513', '522', '523', '602', '603', '612', '613', '622', '623']
The number of cases are: 36


In [12]:
# --- Load All Data into Dictionary ---
data = {}

for xyz in tqdm(available_cases):
    # print(xyz)

    scheme = scheme_map[xyz[0]]
    grid_size = grid_map[xyz[1]]
    precision = precision_map.get(xyz[2], None)
    if precision is None:
        raise AssertionError("No precision provided")

    key = f"{scheme}_G{grid_size}_P{precision}"
    colnames = ["time"] + [f"col{col_num:04d}" for col_num in range(1, grid_size + 2)]

    var_dict = {}
    for var in tqdm([ "dens", "pres", "temp", "u",  # General physical variables
                    "h2", "h", "o", "o2", "oh", "h2o", "ho2", "h2o2", "ar", "n2"]):  # Chemical species
        fname = os.path.join(base_folder, f"{xyz}GT", f"output_sol_{var}_{xyz}GT.csv")
        df = pd.read_csv(fname, header=None, names=colnames, dtype=str).astype(np.longdouble)
        var_dict[var] = df
    data[key] = var_dict

100%|██████████| 36/36 [05:06<00:00,  8.52s/it]


In [14]:
os.makedirs(os.path.join("/media", "ssudhakar", "DATA_10TB", "data_solver", "newrunGT_images_solutions"), exist_ok=True)
os.makedirs(os.path.join("/media", "ssudhakar", "DATA_10TB", "data_solver", "newrunGT_images_stats_max_diff"), exist_ok=True)

images_folder = os.path.join("/media", "ssudhakar", "DATA_10TB", "data_solver", "newrunGT_images_solutions")
stats_folder = os.path.join("/media", "ssudhakar", "DATA_10TB", "data_solver", "newrunGT_images_stats_max_diff")
# for svar_num in tqdm([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]):  # variables physical
#     for x1 in tqdm(['1', '2', '3', '4', '5', '6']):  # schemes
#         for z1 in ['2', '3']:  # precisions
#             for y1 in [1, 2, 3]:  # grid sizes


for xyz in tqdm(available_cases):
    x1, y1, z1 = xyz[0], xyz[1], xyz[2]
    for svar_num in tqdm([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]):  # variables physical
        time = data[mkey[f"{x1}{y1}{z1}"]][svar1[svar_num]].values[:, 0]
        var1 = data[mkey[f"{x1}{y1}{z1}"]][svar1[svar_num]].values[:, 1:-1]
        varref = data_ref[mkey["423"]][svar1[svar_num]].values[:, 1:-1]

        fig, ax = plt.subplots(figsize=(10,3))
        for t in range(0, min(8000, var1.shape[0]), 400):
            ax.plot(grids[int(y1)], var1[t, :], color='blue', label=gridstrs[int(y1)], lw=1)
            ax.plot(grid_512_GT, varref[t, :], color='red', ls='--', label='$Reference$', alpha=0.3)

        ax.ticklabel_format(axis='y', style='sci', useMathText=True, scilimits=(0,0))
        ax.set_title(f"{scheme_map[x1]} Scheme for timestepping\n{precision_map[z1]}-bit precision")
        ax.set_xlabel("Domain")
        ax.set_ylabel(f'{svar2[svar_num]}')
        handles, labels = plt.gca().get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        plt.legend(by_label.values(), by_label.keys())
        plt.grid(visible=True, which='both', color='gray', lw=0.5, ls='--', alpha=0.5, zorder=0)
        plt.tight_layout(pad=0.05)
        plt.savefig(os.path.join(images_folder, f"stdwave_{x1}{y1}{z1}_{svar1[svar_num]}.png"), dpi=300)
        plt.close('all')

100%|██████████| 36/36 [01:34<00:00,  2.61s/it]


In [20]:


for svar_num in tqdm([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]):  # variables physical
    with open(os.path.join(base_folder, f"stats_max_error_{svar1[svar_num]}.txt"), mode='w') as stdwave_stats_file:
        stdwave_stats_file.write("")
    
    for x1 in tqdm(['1', '2', '3', '4', '5', '6']):  # schemes
        for z1 in ['2', '3']:  # precisions
            y1 = '0'
    
            time = data[mkey[f"{x1}{y1}{z1}"]][svar1[svar_num]].values[:, 0]
            var1 = data[mkey[f"{x1}{int(y1)+0}{z1}"]][svar1[svar_num]].values[:, 1:-1]
            var2 = data[mkey[f"{x1}{int(y1)+1}{z1}"]][svar1[svar_num]].values[:, 1:-1]
            var3 = data[mkey[f"{x1}{int(y1)+2}{z1}"]][svar1[svar_num]].values[:, 1:-1]
            varref = data_ref[mkey["423"]][svar1[svar_num]].values[:, 1:-1]

            diff1 = np.abs(np.abs(varref[:min(8000, var1.shape[0]), ::stride512]) - np.abs(var1[:min(8000, var1.shape[0]), :]))
            diff2 = np.abs(np.abs(varref[:min(8000, var2.shape[0]), ::stride512]) - np.abs(var2[:min(8000, var2.shape[0]), ::stride256]))
            diff3 = np.abs(np.abs(varref[:min(8000, var3.shape[0]), ::stride512]) - np.abs(var3[:min(8000, var3.shape[0]), ::stride512]))

            fig, ax = plt.subplots(figsize=(10,3))
            for t in range(0, min(8000, var1.shape[0], var2.shape[0], var3.shape[0]), 400):
                ax.plot(grid_128_GT, diff1[t, :], color='blue', label='$Nx_{128}$')
                ax.plot(grid_128_GT, diff2[t, :], color='green', ls='--', label='$Nx_{256}$')
                ax.plot(grid_128_GT, diff3[t, :], color='red', ls='--', label='$Nx_{512}$')

            ax.ticklabel_format(axis='y', style='sci', useMathText=True)
            ax.set_title(f"{scheme_map[x1]} Scheme for timestepping\n{precision_map[z1]}-bit precision")
            ax.set_xlabel("Domain")
            ax.set_ylabel('Abs. Error = $| \\varepsilon |$')
            handles, labels = plt.gca().get_legend_handles_labels()
            by_label = dict(zip(labels, handles))
            plt.legend(by_label.values(), by_label.keys())
            plt.grid(visible=True, which='both', color='gray', lw=0.5, ls='--', alpha=0.5, zorder=0)
            plt.tight_layout(pad=0.05)
            plt.savefig(os.path.join(stats_folder, f"stdwave_max_error_{x1}_{z1}_{svar1[svar_num]}.png"), dpi=300)
            plt.close('all')

            with open(os.path.join(base_folder, f"stats_max_error_{svar1[svar_num]}.txt"), mode='a') as stdwave_stats_file:
                stdwave_stats_file.write(f"The variable is {svar2[svar_num]}\n")
                stdwave_stats_file.write(f"The reference scheme used is RK4 scheme - 128 bit precision - 1024 grid points\n")
                stdwave_stats_file.write(f"Max difference between ref. and {scheme_map[x1]} scheme with Nx = {grid_map[f'{int(y1)+0}']} points with {precision_map[f'{z1}']} bit precision = {np.nanmax(diff1):.12f}\n")
                stdwave_stats_file.write(f"Max difference between ref. and {scheme_map[x1]} scheme with Nx = {grid_map[f'{int(y1)+1}']} points with {precision_map[f'{z1}']} bit precision = {np.nanmax(diff2):.12f}\n")
                stdwave_stats_file.write(f"Max difference between ref. and {scheme_map[x1]} scheme with Nx = {grid_map[f'{int(y1)+2}']} points with {precision_map[f'{z1}']} bit precision = {np.nanmax(diff3):.12f}\n")
                stdwave_stats_file.write("*******\n*******")
                stdwave_stats_file.write(f"Avg. difference between ref. and {scheme_map[x1]} scheme with Nx = {grid_map[f'{int(y1)+0}']} points with {precision_map[f'{z1}']} bit precision = {np.nanmean(diff1):.12f}\n")
                stdwave_stats_file.write(f"Avg. difference between ref. and {scheme_map[x1]} scheme with Nx = {grid_map[f'{int(y1)+1}']} points with {precision_map[f'{z1}']} bit precision = {np.nanmean(diff2):.12f}\n")
                stdwave_stats_file.write(f"Avg. difference between ref. and {scheme_map[x1]} scheme with Nx = {grid_map[f'{int(y1)+2}']} points with {precision_map[f'{z1}']} bit precision = {np.nanmean(diff3):.12f}\n")
                stdwave_stats_file.write("*******\n*******\n\n\n")

100%|██████████| 14/14 [00:38<00:00,  2.75s/it]
